In [2]:
import sys
# sys.path.append("D:\thesis\thesis\rllib")
import os
import rllib as rl
import json
import joblib
import numpy as np
import pandas as pd
dataset_train = "./data_preprossing/output/train_cleaned.csv"
dataset_test = "./data_preprossing/output/test_cleaned.csv"
symptom_map_file = "./data/basic/symptoms_db.json"
condition = "./data_v2/conditions.json"
condition_db = "./data/basic/conditions_db.json"
data_dir = "./data_v2/"

In [3]:
with open(condition) as fp:
        conditions = json.load(fp)

all_piors={}
for value in conditions.values():
    age_keys = value.get("age")
    gender_keys = value.get("sex")
    incidence = value.get("incidence")
    all_piors[value.get("condition_name")] = {
        "age":{},
        "sex":{},
        "incidence":{}
    }
    for age in age_keys.values():
        all_piors[value.get("condition_name")]["age"][age.get("slug")]=age.get("probability")
    for sex in gender_keys.values():
        all_piors[value.get("condition_name")]["sex"][sex.get("slug")]=sex.get("probability")

    all_piors[value.get("condition_name")]["incidence"]=incidence

all_piors_file = os.path.join(data_dir, "all_piors.json")
with open(all_piors_file, "w") as fp:
    json.dump(all_piors, fp, indent=4)




In [10]:
from fractions import Fraction
piors = "./data/basic/all_piors.json"
with open(piors) as fp:
        all_piors = json.load(fp)
default_age={
"age-0-1-years": 1.536246615603,
"age-1-4-years" : 4.6087398468089,
"age-5-14-years" : 11.4611269862763,
"age-15-29-years" : 21.3235812629491,
"age-30-44-years" :20.2551962493181,
"age-45-59-years" : 20.4205176044076,
"age-60-74-years" : 13.987739661922,
"age-g-75-years" : 6.4068517727151
}
default_sex={
    "sex-male":50,
    "sex-famale":50,
}


def get_pro_age(condition,age):
    value = all_piors.get(condition)
    age_keys = value.get("age")
    age_keys = default_age if age_keys=={} else age_keys
    for a in age_keys.keys():
        ages = a.split("-")
        age_min,age_max = ages[1],ages[2]
        if age_min =="g":
            age_min=age_max
            age_max=140
        age_min,age_max = int(age_min),int(age_max)
        if (age>=age_min and age<=age_max):
            return age_keys[a]
    return 0

p1 = get_pro_age('Pulmonary edema  young',0)/100


def get_pro_sex(condition,sex):
    value = all_piors.get(condition)
    sex_keys = value.get("sex")
    sex_keys = default_sex if sex_keys=={} else sex_keys
    print(sex_keys)
    gender = "sex-male" if sex == '0' else "sex-female"
    return sex_keys[gender]

p2 = get_pro_sex('Pneumococcal pneumonia',1)/100

def get_pro_incidence(condition):
    v = all_piors.get(condition)
    value = v.get("incidence")
    value = value.split('/')
    result = float(value[0])/float(value[1])
    return result
print(get_pro_incidence('Pneumococcal pneumonia'))
ins = get_pro_incidence('Pneumococcal pneumonia')
print(p1*p2+ins)


{'sex-male': 71.4, 'sex-female': 28.6}
0.01015
0.29615


In [6]:
    def get_age(condition):
        import random
        #synthea generated dataset is not based on the age distribution, so we re-set age
        value = all_piors.get(condition)
        ages = value.get("age")
        age_keys = default_age if ages=={} else ages
        age_position = randomS(age_keys)
        age = age_position.split("-")
        age_min,age_max = age[1],age[2]
        if age_min =="g":
            age_min=age_max
            age_max=140
        age_min,age_max = int(age_min),int(age_max)
        return random.randint(age_min,age_max)
    
    def randomS(age_keys):
        import random
        target = random.randint(0,int(sum(age_keys.values())))
        sum_=0
        for k,v in age_keys.items():
            sum_+=v
            if sum_>=target:
                return k
    for i in range(1000):
        age = get_age("IBS (Constipation)")
        print(age)



41
54
49
56
40
61
53
125
54
62
45
63
62
45
42
44
131
54
58
49
54
61
52
46
92
61
108
76
53
106
44
57
100
135
48
59
54
62
39
54
52
105
53
140
63
58
63
108
126
44
126
42
56
57
57
60
58
44
63
118
50
46
59
48
46
63
135
44
42
64
40
54
53
61
130
58
46
43
43
107
59
139
99
51
122
58
120
52
55
55
39
54
60
53
55
64
57
52
56
53
59
44
51
59
53
56
95
56
50
42
55
120
87
55
51
49
38
57
58
64
41
85
42
40
125
48
57
108
120
64
39
53
75
49
48
56
38
58
52
93
100
50
58
61
62
128
35
50
136
61
49
50
62
71
73
60
103
63
42
56
49
126
83
63
64
45
69
41
61
58
63
40
111
49
37
116
91
48
104
41
52
118
51
48
62
39
49
61
64
47
52
42
64
57
54
52
56
90
60
93
44
61
39
103
67
58
52
55
65
36
62
64
61
73
40
52
59
64
101
75
50
37
57
55
63
58
64
54
64
45
104
55
62
35
48
36
63
43
54
64
54
63
105
53
83
52
55
91
87
65
130
55
64
51
84
60
38
39
59
39
43
58
44
91
54
35
44
63
42
48
54
81
57
61
61
59
58
45
63
59
64
84
52
102
59
45
103
53
58
66
49
139
61
57
49
51
63
73
56
60
51
131
35
41
51
52
54
52
100
99
38
53
53
59
51
57
35
114
41
4